In [14]:
# Import Dependencies 
from bs4 import BeautifulSoup
import re
from splinter import Browser
import pandas as pd
import requests

In [2]:
#executable path to driver 
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

**NASA Mars News**

In [3]:
#text splinter
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
#HTML, then parse with BeautifulSoup
html = browser.html
bsoup = BeautifulSoup(html, 'html.parser')

#most recent news title and paragraph
news_title = bsoup.find('div', class_ = 'content_title').find('a')
news_p = bsoup.find('div', class_= 'article_teaser_body').text

#print 
print(news_title)
print(news_p)



<a href="/news/8506/whats-mars-solar-conjunction-and-why-does-it-matter/" target="_self">What's Mars Solar Conjunction, and Why Does It Matter?</a>
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


**JPL Mars Space Images - Featured Image**

In [5]:
#Images splinter
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [20]:
#getting images
page = requests.get('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')
soup = BeautifulSoup(page.content, 'html.parser')
x = soup.find('div', attrs={'class':"carousel_items"}).article['style']

# use RegEx to extract url between the ' '
image_url = (re.findall(r"'(.*?)'",x,re.DOTALL)[0])

#full url
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/' + image_url

**Mars Weather**

In [22]:
# Visit Mars Weather Twitter through splinter module
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [23]:
# HTML Object 
html_weather = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_weather, 'html.parser')

# Find all elements that contain tweets
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')

# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in latest_tweets: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)
pressure at 7.70 hPapic.twitter.com/MhPPOHJg3m


**Mars Facts**

In [74]:
url = "https://space-facts.com/mars/"
table = pd.read_html(url) 

df = table[0]
df1 = df.iloc[:,0:2]
new_df = df1.rename(columns={"Mars-Earth Comparison": "Property", "Mars": "Value"})

new_df.columns = ['Property', 'Value']
new_df

,Property,Value
0,Diameter:,"6,779 km"
1,Mass:,6.39 × 10^23 kg
2,Moons:,2
3,Distance from Sun:,"227,943,824 km"
4,Length of Year:,687 Earth days
5,Temperature:,-153 to 20 °C


In [76]:
mars_facts_html = new_df.to_html(header=False, index=False)

**Mars Hemisphere**

In [77]:
# Visit hemispheres website through splinter module 
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [78]:
# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]